In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import glob
import PIL
import gc
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.models import Sequential

In [2]:
!conda install ../input/how-to-use-pyvips-offline/*.tar.bz2



######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
###########################

In [3]:
import pyvips

In [4]:
Image.MAX_IMAGE_PIXELS = None

In [5]:
train_data = pd.read_csv("../input/mayo-clinic-strip-ai/train.csv")
test_data = pd.read_csv("../input/mayo-clinic-strip-ai/test.csv")
display(train_data)

,image_id,center_id,patient_id,image_num,label
0,006388_0,11,006388,0,CE
1,008e5c_0,11,008e5c,0,CE
2,00c058_0,11,00c058,0,LAA
3,01adc5_0,11,01adc5,0,LAA
4,026c97_0,4,026c97,0,CE
...,...,...,...,...,...
749,fe9645_0,3,fe9645,0,CE
750,fe9bec_0,4,fe9bec,0,LAA
751,ff14e0_0,6,ff14e0,0,CE
752,ffec5c_0,7,ffec5c,0,LAA


In [6]:
train_df = pd.DataFrame(columns=['Path','image_id'])
train_df['Path'] = glob.glob("../input/mayo-convert-tif-to-jpg/train/*")
train_df['image_id'] = train_df['Path'].apply(lambda x: x.split('/')[-1].replace('.jpg',""))
train_df


,Path,image_id
0,../input/mayo-convert-tif-to-jpg/train/c9d53d_...,c9d53d_0
1,../input/mayo-convert-tif-to-jpg/train/0b7cc8_...,0b7cc8_0
2,../input/mayo-convert-tif-to-jpg/train/ea2aad_...,ea2aad_0
3,../input/mayo-convert-tif-to-jpg/train/8cf32b_...,8cf32b_0
4,../input/mayo-convert-tif-to-jpg/train/46def4_...,46def4_0
...,...,...
749,../input/mayo-convert-tif-to-jpg/train/04439c_...,04439c_0
750,../input/mayo-convert-tif-to-jpg/train/56d177_...,56d177_3
751,../input/mayo-convert-tif-to-jpg/train/509042_...,509042_0
752,../input/mayo-convert-tif-to-jpg/train/a26055_...,a26055_0


In [7]:
train_data = train_data.merge(train_df,on='image_id')
train_data

,image_id,center_id,patient_id,image_num,label,Path
0,006388_0,11,006388,0,CE,../input/mayo-convert-tif-to-jpg/train/006388_...
1,008e5c_0,11,008e5c,0,CE,../input/mayo-convert-tif-to-jpg/train/008e5c_...
2,00c058_0,11,00c058,0,LAA,../input/mayo-convert-tif-to-jpg/train/00c058_...
3,01adc5_0,11,01adc5,0,LAA,../input/mayo-convert-tif-to-jpg/train/01adc5_...
4,026c97_0,4,026c97,0,CE,../input/mayo-convert-tif-to-jpg/train/026c97_...
...,...,...,...,...,...,...
749,fe9645_0,3,fe9645,0,CE,../input/mayo-convert-tif-to-jpg/train/fe9645_...
750,fe9bec_0,4,fe9bec,0,LAA,../input/mayo-convert-tif-to-jpg/train/fe9bec_...
751,ff14e0_0,6,ff14e0,0,CE,../input/mayo-convert-tif-to-jpg/train/ff14e0_...
752,ffec5c_0,7,ffec5c,0,LAA,../input/mayo-convert-tif-to-jpg/train/ffec5c_...


In [8]:
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])
train_data


,image_id,center_id,patient_id,image_num,label,Path
0,006388_0,11,006388,0,0,../input/mayo-convert-tif-to-jpg/train/006388_...
1,008e5c_0,11,008e5c,0,0,../input/mayo-convert-tif-to-jpg/train/008e5c_...
2,00c058_0,11,00c058,0,1,../input/mayo-convert-tif-to-jpg/train/00c058_...
3,01adc5_0,11,01adc5,0,1,../input/mayo-convert-tif-to-jpg/train/01adc5_...
4,026c97_0,4,026c97,0,0,../input/mayo-convert-tif-to-jpg/train/026c97_...
...,...,...,...,...,...,...
749,fe9645_0,3,fe9645,0,0,../input/mayo-convert-tif-to-jpg/train/fe9645_...
750,fe9bec_0,4,fe9bec,0,1,../input/mayo-convert-tif-to-jpg/train/fe9bec_...
751,ff14e0_0,6,ff14e0,0,0,../input/mayo-convert-tif-to-jpg/train/ff14e0_...
752,ffec5c_0,7,ffec5c,0,1,../input/mayo-convert-tif-to-jpg/train/ffec5c_...


(Try reducing number of CE data in training )


In [9]:
y = train_data["label"]
train, test = train_test_split(train_data, test_size=0.25, stratify = y)

train = train.reset_index(drop=True)
train

,image_id,center_id,patient_id,image_num,label,Path
0,b7f78e_0,3,b7f78e,0,0,../input/mayo-convert-tif-to-jpg/train/b7f78e_...
1,029c68_0,5,029c68,0,0,../input/mayo-convert-tif-to-jpg/train/029c68_...
2,2f4974_0,11,2f4974,0,0,../input/mayo-convert-tif-to-jpg/train/2f4974_...
3,c1b288_0,11,c1b288,0,0,../input/mayo-convert-tif-to-jpg/train/c1b288_...
4,7f343b_0,11,7f343b,0,0,../input/mayo-convert-tif-to-jpg/train/7f343b_...
...,...,...,...,...,...,...
560,45c6ce_0,11,45c6ce,0,0,../input/mayo-convert-tif-to-jpg/train/45c6ce_...
561,0bd414_0,3,0bd414,0,1,../input/mayo-convert-tif-to-jpg/train/0bd414_...
562,4550fb_0,11,4550fb,0,0,../input/mayo-convert-tif-to-jpg/train/4550fb_...
563,8b90fa_0,11,8b90fa,0,0,../input/mayo-convert-tif-to-jpg/train/8b90fa_...


In [10]:
train['label'].value_counts()

0    410
1    155
Name: label, dtype: int64

Building Custom Generator

In [11]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, df, batch_size=8, n_classes=2, shuffle=False,to_fit=True):
        'Initialization'
        self.df = df
        self.to_fit = to_fit
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.df))
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.augmentor = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )
        self.on_epoch_end()
    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size: (index+1)*self.batch_size]
        X = np.empty((self.batch_size,512,512,3),dtype=np.float32)
        y = np.empty((self.batch_size,1),dtype=np.float32)
        
        if self.to_fit == True:
            
            for i, img_path in enumerate(self.df['Path'].iloc[indexes]):
                img = self.read_img(img_path)
                X[i,] = img
                y[i] = self.df['label'].iloc[indexes[i]]
            X_gen = self.augmentor.flow(X, batch_size=self.batch_size, shuffle=False)
            
            return next(X_gen), y
        
        else:
            for i, img_path in enumerate(self.df['Path'].iloc[indexes]):
                img = self.read_img(img_path)
                X[i,] = img
                
            return X
        
        
    def read_img(self, path):
        image =  pyvips.Image.thumbnail(path,512,height=512,size='force')
        image = np.asarray(image)
        image = image.astype(np.float32)
        return image/255
        

In [12]:
training_generator = DataGenerator(train,shuffle = True)
val_generator = DataGenerator(test,shuffle = True)

In [13]:
input_shape=(512,512,3)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

2022-08-18 08:18:32.474674: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-18 08:18:32.610114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-18 08:18:32.611059: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-18 08:18:32.612853: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
os.makedirs('best_model')

In [16]:
callbacks=[
           tf.keras.callbacks.EarlyStopping(patience=3,monitor='loss'),
         tf.keras.callbacks.ReduceLROnPlateau( monitor='loss',factor=0.1,patience=3, mode='min',min_lr=0.00001),
        tf.keras.callbacks.ModelCheckpoint(filepath='./best_model',monitor = 'val_accuracy', save_best_only=True)
]

In [17]:
history = model.fit(training_generator,
          batch_size=8,
          epochs=5,
          validation_data = val_generator,
          shuffle = True,
          callbacks=callbacks)

2022-08-18 08:18:38.574836: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5


2022-08-18 08:18:44.569522: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


70/70 [==============================] - 288s 4s/step - loss: 18.0398 - accuracy: 0.7750 - val_loss: 0.6814 - val_accuracy: 0.0217


2022-08-18 08:23:26.792508: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-08-18 08:23:27.625697: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2114060288 exceeds 10% of free system memory.
2022-08-18 08:23:29.610296: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2114060288 exceeds 10% of free system memory.
2022-08-18 08:23:31.513298: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2114060288 exceeds 10% of free system memory.


Epoch 2/5
70/70 [==============================] - 275s 4s/step - loss: 0.9129 - accuracy: 0.3375 - val_loss: 0.6624 - val_accuracy: 0.5978


2022-08-18 08:28:31.463513: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2114060288 exceeds 10% of free system memory.
2022-08-18 08:28:33.247437: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2114060288 exceeds 10% of free system memory.


Epoch 3/5
70/70 [==============================] - 272s 4s/step - loss: 0.6482 - accuracy: 0.6250 - val_loss: 0.6234 - val_accuracy: 0.5978
Epoch 4/5
70/70 [==============================] - 273s 4s/step - loss: 0.6343 - accuracy: 0.8821 - val_loss: 0.6466 - val_accuracy: 0.9946
Epoch 5/5
70/70 [==============================] - 276s 4s/step - loss: 0.6299 - accuracy: 0.9857 - val_loss: 0.6166 - val_accuracy: 0.9837


In [18]:
test_data

,image_id,center_id,patient_id,image_num
0,006388_0,11,006388,0
1,008e5c_0,11,008e5c,0
2,00c058_0,11,00c058,0
3,01adc5_0,11,01adc5,0


In [19]:
test_path = glob.glob("../input/mayo-clinic-strip-ai/test/*")
test_df = pd.DataFrame(columns=['Path','image_id'])
test_df['Path'] = test_path
test_df['image_id'] = test_df['Path'].apply(lambda x: x.split('/')[-1].replace('.tif',''))
test_df

,Path,image_id
0,../input/mayo-clinic-strip-ai/test/006388_0.tif,006388_0
1,../input/mayo-clinic-strip-ai/test/00c058_0.tif,00c058_0
2,../input/mayo-clinic-strip-ai/test/008e5c_0.tif,008e5c_0
3,../input/mayo-clinic-strip-ai/test/01adc5_0.tif,01adc5_0


In [20]:
test_data = test_data.merge(test_df,on='image_id')
test_data


,image_id,center_id,patient_id,image_num,Path
0,006388_0,11,006388,0,../input/mayo-clinic-strip-ai/test/006388_0.tif
1,008e5c_0,11,008e5c,0,../input/mayo-clinic-strip-ai/test/008e5c_0.tif
2,00c058_0,11,00c058,0,../input/mayo-clinic-strip-ai/test/00c058_0.tif
3,01adc5_0,11,01adc5,0,../input/mayo-clinic-strip-ai/test/01adc5_0.tif


In [21]:
test_gen = DataGenerator(test_data,batch_size=1,to_fit=False)


In [22]:
#from keras.models import load_model
#saved_model = load_model('./best_model')

In [23]:
predictions = model.predict(test_gen, verbose=1)

4/4 [==============================] - 149s 32s/step


In [24]:
df_preds = pd.DataFrame(predictions, columns=["CE", "LAA"])
df_preds['patient_id'] = test_data['patient_id'] 
df_preds.head()

,CE,LAA,patient_id
0,0.533790,0.466210,006388
1,0.532593,0.467407,008e5c
2,0.535249,0.464751,00c058
3,0.518508,0.481492,01adc5


In [25]:
sub = {'patient_id':df_preds['patient_id'],'CE': df_preds['CE'],'LAA':df_preds['LAA']}
submission = pd.DataFrame(sub)
submission = submission.groupby("patient_id").mean().reset_index()
submission

,patient_id,CE,LAA
0,006388,0.533790,0.466210
1,008e5c,0.532593,0.467407
2,00c058,0.535249,0.464751
3,01adc5,0.518508,0.481492


In [26]:
submission[["patient_id", "CE", "LAA"]].to_csv("/kaggle/working/submission.csv", index=False)
!head submission.csv

patient_id,CE,LAA
006388,0.53379005,0.46620992
008e5c,0.5325929,0.4674071
00c058,0.53524894,0.46475103
01adc5,0.5185083,0.4814917
